In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)
#remove heading
data = data[3:]
#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())
#extract one column --> blocktrace sequence
blocktrace = df[3].tolist()
#convert element to int from str
blocktrace = [int(x) for x in blocktrace]

In [3]:
frame = 3

## FIFO

In [8]:
cache = np.zeros((frame), dtype=int)
hit, miss = 0, 0

#print("block", "\t\t hit", "\t\t miss", "\t\t cache", "\t\tpointer")

pointer = 0
for block in tqdm(blocktrace):
    if block in cache:
        hit += 1

    elif frame - np.count_nonzero(cache) != 0:
        cache[np.where(cache==0)[0][0]] = block
        miss += 1

    else:
        cache[pointer % frame] = block
        pointer += 1
        miss += 1

  #  print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache, "\t\t", pointer)

hitrate = hit / (hit + miss)
print(hitrate)

100%|██████████| 1322897/1322897 [00:05<00:00, 253344.34it/s]

0.048625856737145826


## OPTIMAL

In [9]:
def checkindex(item, ilist):
  
  try:
    return ilist.index(item)
  except ValueError:
    return -1 

In [35]:
def checkindexnp(item, nparray):

    try:
        return np.where(nparray==item)[0][0]
    except IndexError:
        return -1

In [ ]:
cache = np.zeros((frame), dtype=int)

hit, miss = 0, 0

print("block", "\t\t hit", "\t\t miss", "\t\t cache")
for i, block in enumerate(tqdm(blocktrace)):
    if block in cache:
        hit += 1


    elif frame - np.count_nonzero(cache) != 0:
        cache[np.where(cache==0)[0][0]] = block
        miss += 1

    else:
        remain_block_trace = np.array(blocktrace[i:])

        indexed = [checkindexnp(j, remain_block_trace) for j in cache]

        indexed2 = np.array[filter(lambda a: a != -1, indexed)]

        if len(indexed2) == 1:
            random_block_index = np.random.randint(len(cache))
            cache[random_block_index] = block
            if remain_block_trace[indexed2[0]] not in cache:
                cache[random_block_index] = block
                cache.append(remain_block_trace[indexed2[0]])
                cache.append(block)
                miss += 1

            elif len(indexed2) == 0:
                cache.pop(0)
                cache.append(block)
                miss += 1

            else:
                cache.remove(remain_block_trace[max(indexed2)])
                cache.append(block)
                miss += 1


    print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

hitrate = hit / (hit + miss)
print("\n\nhitrate: ", hitrate)

In [4]:
l = [1,2,3,4,3,4,5,3,5,6,7]
l2 = np.array(l)

In [21]:
l2[3]

4

In [20]:
np.delete(l2, np.random.randint(len(l2)))

array([2, 3, 4, 3, 4, 5, 3, 5, 6, 7])